In [1]:
import math
from operator import itemgetter
import time

# 定义save和load模型的函数

In [2]:
import pickle

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

# 定义变量

In [4]:
train_file = 'class1_train.csv'
test_file = 'class1_test.csv'
n_sim_item = 20
n_rec_item = 10
print('Similar item number = %d' % n_sim_item)
print('Recommneded item number = %d' % n_rec_item)
# 将数据集划分为训练集和测试集
trainSet = {}
testSet = {}
# 商品相似度矩阵
itm_sim_matrix = {}
item_popular = {}  # 全局热门商品
item_count = 0
# 读文件，返回文件的每一行
def load_file(filename):
    with open(filename, 'r') as f:
        for i, line in enumerate(f):
            yield line.strip('\r\n')
    print('Load %s success!' % filename)

Similar item number = 20
Recommneded item number = 10


# 加载训练集

In [5]:
t1=time.time()
#加载、解析训练集
trainSet_len = 0
for line in load_file(train_file):
    user, item, clk_count = line.split(',')  # 这里要事先统计出商品被该用户点击的次数
    trainSet.setdefault(user, {})#添加一个用户
    trainSet[user][item] = clk_count#给用户添加一个商品，值为用户对该商品的点击数（连续日期内）
    trainSet_len += 1
print('TrainSet behavior = %s' % trainSet_len)
print('TrainSet users = '+str(len(trainSet)))
t2=time.time()
print('加载训练集用了%fs' % (t2-t1))

Load class1_train.csv success!
TrainSet behavior = 2837067
TrainSet users = 163810
加载训练集用了4.138351s


# 加载测试集

In [6]:
t1=time.time()
#加载、解析测试集
testSet_len = 0
for line in load_file(test_file):
    user, item, clk_count = line.split(',')  # 这里要事先统计出商品被该用户点击的次数
    testSet.setdefault(user, {})
    testSet[user][item] = clk_count
    testSet_len += 1
print('TestSet behavior = %s' % testSet_len)
print('TestSet users = '+str(len(testSet)))
t2=time.time()
print('加载测试集用了%fs'% (t2-t1))

Load class1_test.csv success!
TestSet behavior = 1891708
TestSet users = 163705
加载测试集用了2.366015s


# 计算全局热门商品

In [7]:
t1=time.time()
for user, items in trainSet.items():#对于训练集中的每个用户、该用户点击过的商品列表
    for item in items:#对于商品列表中的每个商品
        if item not in item_popular:#如果没有添加到全局热门商品列表中
            item_popular[item] = 0#添加进去，默认值为0
        item_popular[item] += 1#添加进去之后，用户数+1。

item_count = len(item_popular)#对训练集遍历一遍后，得到所有商品的数量
print("Total item number = %d" % item_count)
t2=time.time()
print('计算全局热门商品%fs'% (t2-t1))

Total item number = 110
计算全局热门商品0.658630s


# 保存全局热门商品

In [8]:
save_obj(item_popular,'class1_popular')

# 计算商品相交用户数矩阵

In [9]:
# 计算出每两个商品相交的用户数
t1=time.time()
# b_count=0
for user, items in trainSet.items():#对每个用户、及他的点击商品列表来说
    for m1 in items:#遍历一次商品列表
        for m2 in items:#第二次遍历商品列表
            if m1 == m2:#自己和自己不用做相似度比较
                continue
            itm_sim_matrix.setdefault(m1, {})#商品相似列表，添加一个商品
            itm_sim_matrix[m1].setdefault(m2, 0)#该商品添加一个相似商品
            itm_sim_matrix[m1][m2] += 1#他们的相似度+1，即有一个人同时喜欢这两个商品
#     b_count+=1
#     print(b_count)
print("Build co-rated users matrix success!")
t2=time.time()
print('计算每两个商品相交的用户数%fs'% (t2-t1))#22.5min --15s

Build co-rated users matrix success!
计算每两个商品相交的用户数32.587328s


# 保存商品相交用户数矩阵

In [10]:
save_obj(itm_sim_matrix,'sim_matrix_class1')

# 根据公式计算商品相似度

In [11]:
# 根据公式计算商品之间的相似性
t1=time.time()
print("Calculating item similarity matrix ...")
# b_count=0
m_len=len(itm_sim_matrix)
print("itm_sim_matrix length = %d" % m_len)
for m1, related_items in itm_sim_matrix.items():#对每个商品，及它相似的商品列表来说
    for m2, count in related_items.items():#对相似商品列表中的每个商品，及它们相交的用户数
        # 注意0向量的处理，即某电影的用户数为0
        if item_popular[m1] == 0 or item_popular[m2] == 0:
            itm_sim_matrix[m1][m2] = 0
        else:
            itm_sim_matrix[m1][m2] = count / math.sqrt(item_popular[m1] * item_popular[m2])
        #减轻了热门物品会和很多物品相似的可能性
#     b_count+=1
#     if b_count%100 == 0 :
#         print('完成%f'%(b_count/m_len*100.0))
print('Calculate item similarity matrix success!')
t2=time.time()
print('计算商品相似度矩阵%fs'% (t2-t1))#16min  --2s

Calculating item similarity matrix ...
itm_sim_matrix length = 109
Calculate item similarity matrix success!
计算商品相似度矩阵0.011968s


In [12]:
t1=time.time()
save_obj(itm_sim_matrix,'class1_sim_matrix')
t2=time.time()
print('保存商品相似度矩阵用了%fs'% (t2-t1))#19.5min

保存商品相似度矩阵用了0.002992s


# 针对目标用户U，找到K个相似的商品，并推荐其N个商品

In [56]:
#找出热门商品列表中的N个商品做备用
K = 100
N = 10
popular_rank=[] 

t1=time.time()
item_popular=load_obj('class1_popular')
popular_rank=sorted(item_popular.items(), key=itemgetter(1), reverse=True)[:N]#根据商品点击人数排序
t2=time.time()
print(popular_rank)
print('获得前N个热门商品用了%fs'% (t2-t1))

[('16', 135056), ('1801', 112711), ('50010788', 102771), ('50006843', 95195), ('50025705', 91392), ('1625', 90288), ('30', 83301), ('50002766', 78134), ('50012029', 69455), ('50006842', 69075)]
获得前N个热门商品用了0.000000s


In [57]:
#找到和用户历史上感兴趣的物品最相似的物品集合
t1=time.time()
itm_sim_matrix=load_obj('class1_sim_matrix')
print(popular_rank)
t2=time.time()
print('加载商品相似矩阵用了%fs'% (t2-t1))

def recommend(user):
    new_user=False
    new_items_count=0
    rank = {}
    if user not in trainSet:#新用户，没有过去点击的信息
        new_user=True
        return popular_rank,new_user,new_items_count #全部推荐热门商品
    else:
        clicked_items = trainSet[user]
        for item, clk_count in clicked_items.items():#对于该用户点击过的商品列表中的每个商品、点击次数
            #找到与item最相似的K个商品
            if item not in itm_sim_matrix:
                new_items_count+=1
                continue
            else:
                for related_item, w in sorted(itm_sim_matrix[item].items(), key=itemgetter(1), reverse=True)[:K]:
                    if related_item in clicked_items:#如果点击过了，就不再推荐
                        continue
                    rank.setdefault(related_item, 0)#推荐列表添加第一个相似商品
                    rank[related_item] += w * float(clk_count)#计算该相似商品的相关度
    #返回排序后的推荐商品列表
    rank_list=sorted(rank.items(), key=itemgetter(1), reverse=True)[:N]
    rank_len=len(rank_list)
    if rank_len<N:
        for i in range(N-rank_len):
            rank_list.append(popular_rank[i])
    return rank_list,new_user,new_items_count

[('16', 135056), ('1801', 112711), ('50010788', 102771), ('50006843', 95195), ('50025705', 91392), ('1625', 90288), ('30', 83301), ('50002766', 78134), ('50012029', 69455), ('50006842', 69075)]
加载商品相似矩阵用了0.003011s


# 对测试集进行测试

In [70]:
#查询有没有我要查的用户：
import pandas as pd

test_df=pd.read_csv('class1_test.csv')
test_df.head()

,user_id,class1_id,clk
0,51309,16,24
1,51309,21,1
2,51309,25,8
3,51309,26,1
4,51309,1512,2


In [97]:
with open('classify_value3.txt', 'r') as f:
    for i, line in enumerate(f):
        line.strip('\r\n')
        items = line.split('\t')
        if 5==int(items[-1]):
            print(items)
            break

['758255951', '6752.5', '313.0', '3.0', '6.0', '0.0', '2', '2', '2', '1', '3190.0', '3290.0', '3290.0', '2990.0', '5\n']


In [30]:
test_df.dtypes

user_id      int64
class1_id    int64
clk          int64
dtype: object

In [117]:
user=758255951
print(len(test_df.loc[test_df['user_id']==user,:]))

13


In [118]:
print('Evaluating start ...')
# for i, user in enumerate(testSet):#遍历测试集的每个用户
#     if i==1:
print('Test User:')
print(str(user))
test_items = testSet.get(str(user), None)    #测试集中用户点击过的商品
print('Test Items:')
print(test_items)
rec_items,new_user,new_item = recommend(user)   #得到推荐的商品及计算出的用户对它们的兴趣
print('Recommend Items:')
print(rec_items)
hit=[]
for item, w in rec_items: #遍历给user推荐的商品
    if item in test_items:    #测试集中有该商品
        hit.append(item)              #推荐命中+1        
print('Hit Items:')
print(hit)
#         break

Evaluating start ...
Test User:
758255951
Test Items:
{'16': '4', '30': '18', '1625': '2', '50006842': '2', '50008165': '2', '50010728': '2', '50011397': '1', '50011699': '25', '50011740': '1', '50012029': '18', '50468001': '1', '50510002': '10', '122650005': '1'}
Recommend Items:
[('16', 135056), ('1801', 112711), ('50010788', 102771), ('50006843', 95195), ('50025705', 91392), ('1625', 90288), ('30', 83301), ('50002766', 78134), ('50012029', 69455), ('50006842', 69075)]
Hit Items:
['16', '1625', '30', '50012029', '50006842']


In [116]:
print('[')
for i in range(6):
    print('[')
    for j in range(47): 
        print('[%d,%d,%d]'%(i,j,0),end='')      
        if j!=46:
            print(',',end='')
    print(']',end='')  
    if i!=5:
        print(',')


[
[
[0,0,0],[0,1,0],[0,2,0],[0,3,0],[0,4,0],[0,5,0],[0,6,0],[0,7,0],[0,8,0],[0,9,0],[0,10,0],[0,11,0],[0,12,0],[0,13,0],[0,14,0],[0,15,0],[0,16,0],[0,17,0],[0,18,0],[0,19,0],[0,20,0],[0,21,0],[0,22,0],[0,23,0],[0,24,0],[0,25,0],[0,26,0],[0,27,0],[0,28,0],[0,29,0],[0,30,0],[0,31,0],[0,32,0],[0,33,0],[0,34,0],[0,35,0],[0,36,0],[0,37,0],[0,38,0],[0,39,0],[0,40,0],[0,41,0],[0,42,0],[0,43,0],[0,44,0],[0,45,0],[0,46,0]],
[
[1,0,0],[1,1,0],[1,2,0],[1,3,0],[1,4,0],[1,5,0],[1,6,0],[1,7,0],[1,8,0],[1,9,0],[1,10,0],[1,11,0],[1,12,0],[1,13,0],[1,14,0],[1,15,0],[1,16,0],[1,17,0],[1,18,0],[1,19,0],[1,20,0],[1,21,0],[1,22,0],[1,23,0],[1,24,0],[1,25,0],[1,26,0],[1,27,0],[1,28,0],[1,29,0],[1,30,0],[1,31,0],[1,32,0],[1,33,0],[1,34,0],[1,35,0],[1,36,0],[1,37,0],[1,38,0],[1,39,0],[1,40,0],[1,41,0],[1,42,0],[1,43,0],[1,44,0],[1,45,0],[1,46,0]],
[
[2,0,0],[2,1,0],[2,2,0],[2,3,0],[2,4,0],[2,5,0],[2,6,0],[2,7,0],[2,8,0],[2,9,0],[2,10,0],[2,11,0],[2,12,0],[2,13,0],[2,14,0],[2,15,0],[2,16,0],[2,17,0],[2,18,0],[

In [114]:
import numpy as np

In [ ]:
np.array()

In [15]:
print('Evaluating start ...')
# 准确率和召回率
hit = 0
rec_count = 0
test_count = 0
# 覆盖率
all_rec_items = set()#不重复的所有推荐商品

new_user_count=0
new_item_count=0

for i, user in enumerate(testSet):#遍历测试集的每个用户
    test_items = testSet.get(user, {})    #测试集中用户点击过的商品
    rec_items,new_user,new_item = recommend(user)   #得到推荐的商品及计算出的用户对它们的兴趣
    new_item_count+=count
    if new_user:
        new_user_count+=1
    
    for item, w in rec_items: #遍历给user推荐的商品
        if item in test_items:    #测试集中有该商品
            hit += 1                #推荐命中+1
        all_rec_items.add(item)
    rec_count += N
    test_count += len(test_items)

precision = hit / (1.0 * rec_count)#总的命中数/总的推荐商品数
recall = hit / (1.0 * test_count)#总的命中数/测试集中用户点击过的商品总数
coverage = len(all_rec_items) / (1.0 * item_count)#不重复的所有推荐商品/训练集中所有商品数
print('precisioin=%.4f\trecall=%.4f\tcoverage=%.4f' % (precision, recall, coverage))
print('new user percentage = %f'% (new_user_count/len(testSet)))
print('new item percentage = %f'% (new_item_count/test_count))

Evaluating start ...
precisioin=0.0341	recall=0.1476	coverage=0.9909
new user percentage = 0.000043
new item percentage = 0.086538
